<a href="https://colab.research.google.com/github/Omars2003/HuggingFace/blob/main/Resumen_en_es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets #para este modelo ocuparemos Amazon_reviews we only will use english and spanish

In [3]:
from datasets import load_dataset



In [ ]:
spanish_dataset = load_dataset("amazon_reviews_multi", "es")
english_dataset = load_dataset("amazon_reviews_multi", "en")


In [5]:
english_dataset 

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

we are interested in is contained in the review_body and review_title columns

In [ ]:
def muestra_ejemplos(dataset, num_samples=4, seed=50):#seed=un grupo de 50
    ejemplos = dataset["train"].shuffle(seed=seed).select(range(num_samples)) #shuffle funtion cambia el orden 
    for example in ejemplos:
        print(f"\n'>> Title: {example['review_title']}'")
        print(f"'>> Review: {example['review_body']}'")


muestra_ejemplos(english_dataset)
muestra_ejemplos(spanish_dataset)

 Training a summarization model on all 400,000 reviews would take far too long on a single GPU, so instead we’ll focus on generating summaries for a single domain of products.

To get a feel for what domains we can choose from, let’s convert english_dataset to a pandas.DataFrame and compute the number of reviews per product category:

In [20]:
english_dataset.set_format("arrow") #arrow or #pandas
english_df = english_dataset["train"][:]
# Show counts for top 20 products
english_df["product_category"].value_counts()[:20]

-- is_valid: all not null
-- child 0 type: string
  [
    "furniture",
    "home_improvement",
    "home",
    "wireless",
    "pc",
    "industrial_supplies",
    "kitchen",
    "apparel",
    "automotive",
    "camera",
    "lawn_and_garden",
    "watch",
    "beauty",
    "pet_products",
    "drugstore",
    "electronics",
    "toy",
    "digital_ebook_purchase",
    "book",
    "jewelry"
  ]
-- child 1 type: int64
  [
    2984,
    7136,
    17679,
    15717,
    6401,
    1994,
    10382,
    15951,
    7506,
    2139,
    7327,
    761,
    12091,
    7082,
    11730,
    6186,
    8745,
    6749,
    3756,
    2747
  ]

we are going to use the books and e-books reviews, so we need to separate their reviews.

Now when we apply this function to english_dataset and spanish_dataset, the result will contain just those rows involving the book categories. Before applying the filter, let’s switch the format of english_dataset from "pandas" back to "arrow":

In [21]:
def filtros_books(example):
    return (
        example["product_category"] == "book"
        or example["product_category"] == "digital_ebook_purchase"
    )

In [25]:
english_dataset.reset_format()
spanish_dataset.reset_format()

In [ ]:
spanish_books = spanish_dataset.filter(filtros_books)
english_books = english_dataset.filter(filtros_books)
muestra_ejemplos(english_books)
muestra_ejemplos(spanish_books)

WE HAVE a last step about data and itscombining the English and Spanish reviews as a single DatasetDict objec

oncatenate_datasets() function that (as the name suggests) will stack two Dataset objects on top of each other. So, to create our bilingual dataset, we’ll loop over each split, concatenate the datasets for that split, and shuffle the result to ensure our model doesn’t overfit to a single language: